# Summary
- ?? high-level summary of conclusions
- ?? report objectives etc??




# Objectives & Approach

This report describes the findings from an analysis of sales review data for a US retail chain, JCPenney. It is the final assignment in the module: ITNPBD2,  Representing and Manipulating Data and uses several datsets provided, with some suplimentary data to augment this.

The report is structured into four sections:
- Summary
- Data Collation, Exploration & Preparation
- Visualation & Initial Analysis
- Conclusions

### Approach

This has primarily been written as a business report, with conclusions and supporting details. However, it also provides details of the analysis approach taken and listings of the Python code used. In order, not to disrupt the flow of the reporting, as much as is possible, the Python code has been put towards the end of report sections **??? or in appendices ??**.

I was not totally clear about the relative importance of the assignment objectives and so have attempted to meet the objectives stated below, in order of priority:
- Provide a business report to senior managers who are not interested in the detailed approach or source data
- Demonstrate that a data science process (here CRISP-DM) has been followed
- Demonstrate the use of Python for data manipulation and analysis

The original source and timing for the provided datasets is not known and so the following assumptions have been made
- Source is from JCP's operational stock and sales systems
- The data was extracted in Q3 2023

***!! your code must be working, correct, and well commented and shows an appreciation of style, efficiency and reliability. All choices for methods and structures are concisely justified and alternatives are given well thought considerations.***

### CRISP-DM

The approach to the analysis follows the first four parts of the six stage "CRoss-Industry Standard Process for Data Mining' (CRISP-DM) process. ***[!! See citations url and (Ncr et al., 1999) !!]***. In summary, this process is:

1. Business Understanding: Define project objectives and requirements by collaborating with stakeholders
2. Data Understanding: Collect and explore data, analyzing its characteristics and quality
3. Data Preparation: Clean, handle missing values, and transform variables to create a structured dataset
4. Modeling: Apply various techniques such as machine learning algorithms or statistical models to the prepared data
5. Evaluation: Rigorously assess models based on predefined criteria, including performance and reliability
6. Deployment: Integrate successful models into existing systems and monitor their effectiveness

### Who Are JCPenney?

JCPenney (JCP) is a major North American department store chain [Wikipedia](https://en.wikipedia.org/wiki/JCPenney), operating as Penney OpCo LLC. JCP has 656 stores in 49 states plus Puerto Rico according to the [JCP Store Locator](https://www.jcpenney.com/locations/index.html). 

In 2020, JCP filed for bankruptcy and were purchased by an asset management company, a large number of stores were also closed. Later, in August 2023, JCP announced a major turnaround plan to replace its current website and inventory management systems, as well as make major upgardes to its retails stores. See this article for more details: [Modern Retail Sep 2023](https://www.modernretail.co/operations/jcpenney-is-the-latest-department-store-to-announce-a-major-turnaround-plan/).


# Data Collation, Exploration & Preparation

In [1]:
# General setup and imports used throughout the Jupyter Notebook
#
# Libraries For file handling and dataframes
import os
import json
from IPython.display import display
import pandas as pd 

# Libraries ....
## cccccc

# Variables used throughout the notebook
DATA_DIRECTORY = 'JCPenney_Data_Original'  # Designated data folder within the current working directory
AUGMENTED_DATA = 'Data_Additional'         # Additional data sources
 
# A simple utility function to obtain and summarise key elements of a provided dataframe
#
def print_file_summary(data_frame):
    # Create a temporary df and ensure no lists remain, so that unique items can be identified for uniquness
    temp_df = data_frame.copy()
    temp_df = temp_df.map(lambda cell: str(cell) if isinstance(cell, list) else cell)
    
    # Calculate some 
    summary_of_df = pd.DataFrame({'Count': data_frame.count(),
                                 'Missing': data_frame.isnull().sum(), 'Empty': 0,
                                 'Unique': temp_df.nunique(),
                                 'Type': data_frame.dtypes, 
                                 'String': 0, 'Int': 0, 'Float': 0, 'List': 0
                                 })
    summary_of_df['Empty'] = (data_frame == '').sum()
    summary_of_df['String'] = data_frame.map(lambda cell: isinstance(cell, str)).sum()
    summary_of_df['Int'] = data_frame.map(lambda cell: isinstance(cell, int)).sum()
    summary_of_df['Float'] = data_frame.map(lambda cell: isinstance(cell, float)).sum()
    summary_of_df['List'] = data_frame.map(lambda cell: isinstance(cell, list)).sum()

    display(summary_of_df)

### TO DO
- 1. Data Exploration - Explore the data and show you understand its structure and relations
- 2. Data Validation - Check the quality of the data. Is it complete? Are there obvious errors?
- x. Data Preparation - Addresss the issues identified, supliment/augment the data, restructure
    - Select Data - which to use and which to exclude .. with reasoning
    - Clean Data - 'correct, impute, remove' ...
    - Construct Data - derive new attributes as needed/helpful
    - Integrate Data - new datasets, augment, other sources
    - Format Data - reformat as needed, eg string to numeric, dates, categorical  

- ?? data structure, size
- ?? data quality, missing etc

#### Data Completeness
 - Missing Data - Identify and resolve by removal or inferring. Imputation and visualisation, examining descriptive stats
- Noisy - Random errors, eg from faulty sensors, data transmission
- Duplicates - Identify and eliminate duplicates, redundancy. NB: Can occur after data integration
- Inconsistency - Data items don't align, eg DOB and age

## Provided Data Sources & Content

The provided data sources for this analysis of JC Penney consists of two JSON files and three CSV files:
- JSON: jcpenney_products, jcpenney_reviewers
- CSV: products, reviews, users

It was not immediately obvious what the relationships between the two types of data were but the json and CSV files appear to be partial duplicates of each other; also the three CSV files hold slightly less information (eg sales price is missing from the csv files). The CSV files appear to be a first attempt to extract data from the json files (eg the json products file has a JSON field holding multiple user reviews and this has looks to have been extracted to prepare the reviews.csv file).

Given the above, the approach used in this analysis was to go back to the 'orginal' JSON files and work from these but with a sanity check against the three CSV files to make sure no data was missed or inconsistent.

### Data Sources Overview - JSON

It is assumed that the data is a snapshot extract of sales information from JCP databases and the bulk of this has been flattened and used to create the jcpenney_products.json file with the jcpenney_reviewers.json file providing details of individual customers.

The two tables below show the data items and key counts for each file.

In [2]:
# Load the JSON product file and examine the format and content
# NB: Use pandas json load to directly create a dataframe

# Products file source
file_name = 'jcpenney_products.json'
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, file_name)
if not os.path.isfile(file_path):
    raise Exception(f'File not found: {file_path}')

# File load into a Pandas dataframe, retained and not amended
source_jcp_products_df = pd.read_json(file_path, lines=True)
                    
# Initial look at the file and data fields
print(f'File Summary for: {file_name}')
print_file_summary(source_jcp_products_df)
print(f'First 3 Rows')
display(source_jcp_products_df.head(3))

File Summary for: jcpenney_products.json


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
uniq_id,7982,0,0,7982,object,7982,0,0,0
sku,7982,0,67,6044,object,7982,0,0,0
name_title,7982,0,0,6002,object,7982,0,0,0
description,7982,0,543,5620,object,7982,0,0,0
list_price,7982,0,2166,1037,object,7982,0,0,0
sale_price,7982,0,18,2063,object,7982,0,0,0
category,7982,0,636,1169,object,7982,0,0,0
category_tree,7982,0,636,1997,object,7982,0,0,0
average_product_rating,7982,0,0,153,float64,0,0,7982,0
product_url,7982,0,0,7982,object,7982,0,0,0


First 3 Rows


,uniq_id,sku,name_title,description,list_price,sale_price,category,category_tree,average_product_rating,product_url,product_image_urls,brand,total_number_reviews,Reviews,Bought With
0,b6c0b6bea69c722939585baeac73c13d,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'fsdv4141', 'Review': 'You never hav...","[898e42fe937a33e8ce5e900ca7a4d924, 8c02c262567..."
1,93e5272c51d8cce02597e3ce67b7ad0a,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,3.000,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'tpcu2211', 'Review': 'You never hav...","[bc9ab3406dcaa84a123b9da862e6367d, 18eb69e8fc2..."
2,013e320f2f2ec0cf5b3ff5418d688528,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,view all,jcpenney|women|view all,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'pcfg3234', 'Review': 'You never hav...","[3ce70f519a9cfdd85cdbdecd358e5347, b0295c96d2b..."


In [3]:
# Load the JSON reviewers file and examine the format and content
# NB: Use pandas json load to directly create a dataframe

# Reviewers file source
file_name = 'jcpenney_reviewers.json'
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, file_name)
if not os.path.isfile(file_path):
    raise Exception(f'File not found: {file_path}')

# File load into a Pandas dataframe, retained and not amended
source_jcp_reviewers_df = pd.read_json(file_path, lines=True)
                    
# Initial look at the file and data fields
print(f'File Summary for: {file_name}')
print_file_summary(source_jcp_reviewers_df)
print(f'First 3 Rows')
display(source_jcp_reviewers_df.head(3))

File Summary for: jcpenney_reviewers.json


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
Username,5000,0,0,4999,object,5000,0,0,0
DOB,5000,0,0,52,object,5000,0,0,0
State,5000,0,0,57,object,5000,0,0,0
Reviewed,5000,0,0,4030,object,0,0,0,5000


First 3 Rows


,Username,DOB,State,Reviewed
0,bkpn1412,31.07.1983,Oregon,[cea76118f6a9110a893de2b7654319c0]
1,gqjs4414,27.07.1998,Massachusetts,[fa04fe6c0dd5189f54fe600838da43d3]
2,eehe1434,08.08.1950,Idaho,[]


### Data Sources Overview - CSV

***TODO: Decode and compare to JSON***


## Collation, Augmentation & Data Structure

### Data Structure Summary

Having examined the source data (more details and explanation in the sections below) and the areas benefiting augmentation, the following data structure was prepared ready for analysis:
- Sales Activity - From the 7982 rows in the file jcpenney_products.json
- Stock / Product Details - From the 6044 unique items in the file jcpenney_products.json
- Customer Sales Reviews - XXXX unique items in the file jcpenney_products.json
- States & Territories - List of the 57 US states and territories, with population and store totals. Augmented data from JCP and US Census. Loaded from the file JCP_Stores_State_Collated.csv
- Customer Details - List of 5000 customers. Loaded from the file jcpenney_reviewers.json


### Sales Activity


### Stock / Product Details


### Customer Sales Reviews


### States & Territories

Additional information collated and loaded for all US states and territories. Used to validate provided state names and augment the data provided to assist later analysis. Contains 51 states and 6 territories.

The data was sourced from:
- JCP's store locator, see [website](https://www.jcpenney.com/locations/index.html)
- US Census Bureau, see [website](https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html)

#### Data Content

After review and vailidation the resulting dataframe, states_df, consists of:
- territory_flag - Indicates whether a state or a territory
- state_ISO - ISO code of the state, territory
- state_name - Name of the state, territory
- population - Population at 2023
- stores_total - Total number of stores at November 2024

#### Collation & Validation

Loaded from the JCP_Stores_State_Collated.csv file and retained for use in validation and analysis of the JCP customer data.


In [4]:
# Establish a list of states/territories with additional data to augment

# Load the states .csv file, exit if do not exist or are invalid
file_path = os.path.join(os.getcwd(), AUGMENTED_DATA, 'JCP_Stores_State_Collated.csv')
if not os.path.isfile(file_path):
    raise Exception(f"File not found: {file_path}")
states_df = pd.read_csv(file_path)

# Initial look at the file and data fields
print(f'Summary of States - CSV')
print_file_summary(states_df)
print(f'First 3 rows')
display(states_df.head(3))

# Rename column names & set the index on ISO
states_df = states_df.rename(columns={'State or Territory': 'territory_flag', 
                                            'State_ISO': 'state_ISO', 'State_Name': 'state_name',
                                            'Population_2023': 'population',
                                            'Store_Count': 'stores_total'})
#states_df.set_index(keys='state_ISO', inplace=True)

# Convert population to int
states_df['population'] = states_df['population'].str.replace(',', '').astype(int)

# Final look at the file and data fields
print(f'Summary of States - CSV')
print_file_summary(states_df)
print(f'First 3 rows')
display(states_df.head(3))

# Tidy up
del file_path

# Provide a simple state lookup of ISO for a given name
def get_state(state_name):
    matched_state = states_df.loc[states_df['state_name'] == state_name]
    if len(matched_state) == 1:
        return matched_state.iloc[0]
    else:
        return None


Summary of States - CSV


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
State or Territory,57,0,0,2,object,57,0,0,0
State_ISO,57,0,0,57,object,57,0,0,0
State_Name,57,0,0,57,object,57,0,0,0
Population_2023,57,0,0,57,object,57,0,0,0
Store_Count,57,0,0,26,int64,0,57,0,0


First 3 rows


,State or Territory,State_ISO,State_Name,Population_2023,Store_Count
0,State,AL,Alabama,"5,108,468",9
1,State,AK,Alaska,"733,406",1
2,State,AZ,Arizona,"7,431,344",17


Summary of States - CSV


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
territory_flag,57,0,0,2,object,57,0,0,0
state_ISO,57,0,0,57,object,57,0,0,0
state_name,57,0,0,57,object,57,0,0,0
population,57,0,0,57,int64,0,57,0,0
stores_total,57,0,0,26,int64,0,57,0,0


First 3 rows


,territory_flag,state_ISO,state_name,population,stores_total
0,State,AL,Alabama,5108468,9
1,State,AK,Alaska,733406,1
2,State,AZ,Arizona,7431344,17


### Customer Details

Details of customers that have completed a review of a purchase made. With 5,000 unique customer records.

#### Data Content

After review and vailidation the resulting dataframe, customers_df, consists of:
- customer_id - Unique alphanumeric id
- DOB - Date of birth
- state_ISO - ISO code for the state or territory. NB used as a key to the states dataframe
- uniq_id_list - List of uniq_id to cross-reference back to the sales activity records
    ***4030 unique vs 5000, also appear to be some empty lists***

#### Collation & Validation

The jcpenney_reviewers.json file was examined. As these appears to be detailig customers that have completed a review, the term 'customer' was used instead of reviewer.

The following actions were taken:
- Fields Rename: Columns renamed to be consistent with other dataframes
- Duplicates: One customer_id was used twice. To preserve information, it was decided to keep the duplicates and give them a new unique customer_id
- Date of Birth: Surprisingly for 5,000 customers only 52 birth dates were found. Closer examination revealed that a day, month sequence was incremented accross years; with the dates range only being from 26 July to 8 August. The dates appear to be artificially generated. Arguably this field should be dropped as it will not provide any meaningful results. However, it has been converted to a date field and retained purely so that it can be used to demonstrate later analysis
- States: When validating against the states reference file to obtain ISO codes, 187 customers did not match due to the incorrect naming of the US Virgin Islands and US Minor Outlying Islands, so these were corrected. Only the ISO code was retained and the full state name dropped, in preference to it being looked up when required


????
- ***4999 in the CSV file what have they done with the duplicate***



In [6]:
# Establish a customer details dataframe 

# Create a new dataframe for all cutomer reviews
customers_df = source_jcp_reviewers_df.copy()

# Rename customer column names and validate content for each
customers_df = customers_df.rename(columns={'Username': 'customer_id', 
                                            'State': 'state_name',
                                            'Reviewed': 'uniq_id_list'})

# Print the file and data fields
print(f'Summary for customers')
print_file_summary(customers_df)
print(f'First 3 rows - Renamed Columns')
display(customers_df.head(3))

# Identify duplicate customers 
duplicates_flag = customers_df.duplicated(subset=['customer_id'], keep=False)
duplicated = customers_df[duplicates_flag]
print(f'Duplicated Customers:')
display(duplicated)

# Replace duplicates with new customer_id 'DUPnnnxxxxxxx' to preserve 
# Use itertuples as faster for larger datasets
dup_count = 0
for row in duplicated.itertuples():   
    dup_count += 1
    new_id = 'DUP' + str(dup_count).zfill(3) + row.customer_id     
    customers_df.at[row.Index, 'customer_id'] = new_id

# Double check no duplicates remain
duplicates_flag = customers_df.duplicated(subset=['customer_id'], keep=False)
duplicated = customers_df[duplicates_flag]
print(f'Double-Check No Remaining Duplicated Customers:')
display(duplicated)

# DOB convert to date format and examine the dates used
customers_df['DOB'] = pd.to_datetime(customers_df['DOB'], dayfirst=True, errors='coerce')
dates = customers_df.groupby('DOB').size().reset_index(name='counts')
print(f'Dates Count:')
display(dates)
# Drop the date as looks artifically generated and so of no real use in later analysis
# customers_df = customers_df.drop('DOB', axis=1)

# States validation - lookup ISO codes, add to customer data and check for invalid matches
customers_df['state_ISO'] = customers_df['state_name'].apply(lambda x: get_state(x)['state_ISO'] if get_state(x) is not None else None)
unmatched_states = customers_df[customers_df['state_ISO'].isnull()]
print(f'Unmatched States:')
display(unmatched_states[['customer_id', 'state_name']])

# Names mismatch for US Virgin Islands and US Minor Outlying Islands
customers_df.replace('U.S. Virgin Islands', 'US Virgin Islands', inplace=True)
customers_df.replace('Minor Outlying Islands', 'US Minor Outlying Islands', inplace=True)

# Repeat the checks & drop state_name if all ISO populated
customers_df['state_ISO'] = customers_df['state_name'].apply(lambda x: get_state(x)['state_ISO'] if get_state(x) is not None else None)
unmatched_states = customers_df[customers_df['state_ISO'].isnull()]
print(f'Unmatched States:')
display(unmatched_states[['customer_id', 'state_name']])

# Drop the state name, rely on the ISO code and states lookup
if len(unmatched_states) != 0:
    raise Exception(f'Cannot match: {len(unmatched_states)} states')
customers_df = customers_df.drop('state_name', axis=1)

# Visual check on state details
states = customers_df.groupby('state_ISO').size().reset_index(name='counts')
print(f'Customers by State:')
display(states)

# Reviewed validate
# TODO: x-check these to sales activity and to reviews to make sure consistent

# Tidy up
del duplicates_flag, duplicated, dup_count, new_id, row
del dates
del unmatched_states, states


Summary for customers


,Count,Missing,Empty,Unique,Type,String,Int,Float,List
customer_id,5000,0,0,4999,object,5000,0,0,0
DOB,5000,0,0,52,object,5000,0,0,0
state_name,5000,0,0,57,object,5000,0,0,0
uniq_id_list,5000,0,0,4030,object,0,0,0,5000


First 3 rows - Renamed Columns


,customer_id,DOB,state_name,uniq_id_list
0,bkpn1412,31.07.1983,Oregon,[cea76118f6a9110a893de2b7654319c0]
1,gqjs4414,27.07.1998,Massachusetts,[fa04fe6c0dd5189f54fe600838da43d3]
2,eehe1434,08.08.1950,Idaho,[]


Duplicated Customers:


...

,customer_id,state_name
29,wjfh4432,Minor Outlying Islands
104,ulkz1412,Minor Outlying Islands
106,bsqg4331,Minor Outlying Islands
203,bbiv3413,Minor Outlying Islands
215,surt1311,U.S. Virgin Islands
...,...,...
4872,ypcn2342,U.S. Virgin Islands
4940,lric2324,U.S. Virgin Islands
4960,okun1224,Minor Outlying Islands
4970,kjgm1311,U.S. Virgin Islands


Unmatched States:


,customer_id,state_name


Customers by State:


,state_ISO,counts
0,AK,94
1,AL,95
2,AR,92
3,AS,86
4,AZ,71
5,CA,99
6,CO,85
7,CT,82
8,DC,83
9,DE,106


### Data Structure


Sales Activity - 7982 rows in the file jcpenney_products.json
- uniq_id: this uniquely identifies each of the 7982 rows. A random string
- list_price: assumed to be the price at the point of sale in $
    ***?? prices consistent accross channels, any pattern?, 18 missing ***
    *** convert to numeric**
- category_tree: breakdown of the stock catgeorisation eg 'jcpenny|women|skechers' 
- category: lowest level of category 
    *** drop this? why unique 1169 vs 1997 for the treee also ?? not same match to SKU etc eg the use of 'view all' ?? ***
- average_product_rating: rating 1 to 5 
    *** a float with some wierd values .. should be categorical? convert? ***,
    *** is this a calculated average from individual reviews? check***
- product_url: link to website product details 
    *** 7982 And these are unique to the review, not the product ... why? ***
    *** need us vpn to use this? ***
- product_image_urls: link to product image 
    *** 6519, more images than product number, less than product URL ***
- Bought with: series of items linked using the uniq_id 
    *** check these link up? and rename ***
    ***what is is useful for?***

Stock / Product - 6044 unique items in the file jcpenney_products.json
- sku: a unique 'Stock Keeping Unit', format 'xxnnnnnnnnnn', 6044 discrete values
    ***?? are all references valid format, 67 seem odd or blank or not standard format***
- name_title: name of the stock item
    ***?? uniques count about 38 less than SKU***
- description: long description of the item
    ***?? uniques count about 400 less than name***
- list_price: assumed to be the standard item price in $
    ***?? prices appear to be different for some stock, 2166 missing ***
    *** convert to numeric**
- brand: manufacturer brand name eg 'Alfred Dunner'

Customer Reviews - unique items in the file jcpenney_products.json
- total_number_reviews: number of customer reviews for each uniq_id / Sales Channel Activity
    *** is this needed, or just for a cross-check later? convert to numeric**
- Reviews: a json item with several customer reviews, each with: user, review text, score (1 to 5) 



#### Links
?? Link/Cross-Ref
- 'Stock Keeping Unit' (https://en.wikipedia.org/wiki/Stock_keeping_unit)

### Sales Activity



### Stock / Products




### Customer Reviews

In the "jcpenney_products.json" file the series of JSON encoded review details were extracted and a new customer reviews dataframe was created, with individual review records linked back to the sales activiy using the uniq_id. The table below summarises the fields decoded and key counts.

There are a total of 39,063 reviews but only 29,464 appear to be unique review comments. Further analysis found that 15,535 (40%) of reviews were used by several customers, worst case being several instances of 18 customers using the same comments. This could be because the sample data has been automaticaly generated or that customer ids are being created to generate false reviews. This data has not been dropped from the dataset, although later sentiment analysis of the reviews could be misleading.

The scores for reviews in the CSV file have a large number of zero vslues (11,265 out of 39,063) and a quick examination showed that many scores differ between the JSON and CSV source. Therefore the "reviews.CSV" data source was rejected and only the JSON source was used.

In [ ]:
# Establish an initial customer reviews dataframe
# By extracting the series of JSON reviews originaly in the jcpenney_products.json 

# Create a new dataframe for all cutomer reviews
customer_reviews_df = pd.DataFrame()

# Iterate through all rows of the orgonal products data, to extract and decode the series of JSON data
# Create customer review rows, each using a foreign key uniq_id for the relevant sales activity
# TODO: This takes 4 seconds to run, replace with a more efficient approach
for next_row in source_jcp_products_df.itertuples(index=False):
    temp_reviews = next_row.Reviews
    #print(f'UI: {next_row.uniq_id}, {temp_reviews}, {type(temp_reviews)}')
    temp_dict_string = json.dumps(temp_reviews)
    temp_reviews_df = pd.DataFrame(json.loads(temp_dict_string))
    temp_reviews_df.insert(0, 'uniq_id', next_row.uniq_id)
    #print(temp_reviews_df)
    customer_reviews_df = pd.concat([customer_reviews_df, temp_reviews_df])

# Cross-check the customer_id to the customers data frame to make sure all exist
if not customer_reviews_df['User'].isin(customers_df['customer_id']).all():
    print(f'Error: Not all customers setup in customers list')

# Tidy up
del temp_reviews
del temp_dict_string
del temp_reviews_df

# Rename customer column names and validate content for each
customer_reviews_df = customer_reviews_df.rename(columns={'User': 'customer_id', 
                                                          'Review': 'review', 'Score': 'score'})

# Initial look at the file and data fields
print(f'Summary for customer reviews')
print_file_summary(customer_reviews_df)
print(f'First 3 rows')
display(customer_reviews_df.head(3))

# ?? create products ... sales activity file without reviews
# Ensure a valid customer record exists in the customers dataframe



In [ ]:
# Look at how many reviews are duplicates and how many customers are linked to these

duplicates_by_customer = customer_reviews_df.groupby('review')['customer_id'].size().reset_index(name='cust_count')
reviews_duplicated = duplicates_by_customer.groupby('cust_count').count().reset_index()

count_reviews_single = reviews_duplicated[reviews_duplicated['cust_count'] == 1]
count_reviews_duplicated = len(customer_reviews_df) - count_reviews_single['review'].sum()
max_duplicates = reviews_duplicated['cust_count'].max()

print(f'Out of a total of {len(customer_reviews_df)} reviews {count_reviews_duplicated} are duplicates.')
print(f'Or approximately {((count_reviews_duplicated/len(customer_reviews_df)) * 100):.0f}%')
print(f'Several worst case situations with {max_duplicates} customers using the same review comments.')

# Tidy up
del duplicates_by_customer
del reviews_duplicated
del count_reviews_single
del count_reviews_duplicated
del max_duplicates


In [ ]:
# Load the CSV reviews file to cross-check against the data extracted from the JSON sourced reviews

# Load the reviews .csv file, exit if do not exist or are invalid
file_path = os.path.join(os.getcwd(), DATA_DIRECTORY, 'reviews.csv')
if not os.path.isfile(file_path):
    raise Exception(f"File not found: {file_path}")
source_reviewsCSV_df = pd.read_csv(file_path)

# Initial look at the file and data fields
print(f'Summary for customer reviews - CSV')
print_file_summary(source_reviewsCSV_df)
print(f'First 3 rows')
display(source_reviewsCSV_df.head(3))

# Scores look very different

count_zero_scores = source_reviewsCSV_df[source_reviewsCSV_df['Score'] == 0]['Score'].count()
count_zero_scoresJSON = customer_reviews_df[customer_reviews_df['score'] == 0]['score'].count()

print(f'Compare JSON sourced review vs CSV file source')
print(f'Count:  {len(customer_reviews_df)} vs {len(source_reviewsCSV_df)}')
print(f'Scores with zero: {count_zero_scoresJSON:.0f} vs {count_zero_scores:.0f}')
print(f'Mean:  {customer_reviews_df['score'].mean():.1f} vs {source_reviewsCSV_df['Score'].mean():.1f}')


In [ ]:
# Sentiment analysis of reviews

import nltk
#nltk.download()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_categorise(sentiment):
    '''
    Positive: Compound score >= 0.05
    Neutral: Compound score > -0.05 and < 0.05
    Negative: Compound score <= -0.05
    '''
    if sentiment <= -0.05:
        return 1
    elif sentiment < 0.05:
        return 3
    else:
        return 5

review_sentiments = [analyser.polarity_scores(review_text)['compound'] for review_text in customer_reviews_df['review']]
customer_reviews_df['sentimentC'] = [sentiment_categorise(x) for x in review_sentiments]
customer_reviews_df['sentiment'] = review_sentiments

# TODO: Appears to be an inconsistent use of score, some appear to have 1 as best whilst others using 5 as best
# TODO: Classify sentiment anlaysis as good, bad, neutral or 1 3 5? Correlate/fit with score? Evidence to support above?


### Observations


- 4993 users x-check

# Data Visualisation

 3. Data Visualisation - Gain an overall understanding of the data with visualisations

- Initial review, plots etc
- Initial observations, insights

# Data Analysis

 4. Data Analysis = Set some questions and use the data to answer them
 5. Data Augmentation - Add new data from another source to bring new insights to the data you already have

# References

See the CRISP-DM in the intro
- https://www.datascience-pm.com/crisp-dm-2/
- (Ncr et al., 1999) ........ Ncr, P.C. et al. (1999) ‘CRISP-DM 1.0’.